# Lab Assignment Two: Exploring Image Data 

#### Everett Cienkus, Blake Miller, Colin Weil

### 1. Business Understanding


### 2. Data Preparation 

#### 2.1 Read In Images as Numpy Arrays

#### 2.2 Linearize Images

#### 2.3 Visualize Images

### 3. Data Reduction

#### 3.1 Linear dimensionality reduction PCA

#### 3.2 randomized principle components analysis

#### 3.3 Compare PCA and Randomized PCA

#### 3.4 feature extraction

In [ ]:
# explanation of feature extraction

### 4. DAISY